In [11]:
import cv2 as cv
import mediapipe as mp
import copy
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import os
from datetime import date, datetime

In [12]:
def calc_bounding_rect(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_array = np.empty((0, 2), int)

    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)

        landmark_point = [np.array((landmark_x, landmark_y))]

        landmark_array = np.append(landmark_array, landmark_point, axis=0)

    x, y, w, h = cv.boundingRect(landmark_array)

    return [x, y, x + w, y + h]

In [13]:
#загрузка нашей модели
reconstructed_model = keras.models.load_model("my_model.keras")

cap_device = 0
cap_width = 1920
cap_height = 1080
a = True
use_brect = True

In [14]:
#cap = cv.VideoCapture(cap_device, cv.CAP_DSHOW)
cap = cv.VideoCapture('D:\\ML\\Attention-focus-detection-daniyar\\data\\video.mp4')

In [15]:
window_name = 'Мониторинг фокусировки внимания, команда Данияра и Максима'

cv.namedWindow(window_name, cv.WND_PROP_FULLSCREEN)
cv.setWindowProperty(window_name, cv.WINDOW_FULLSCREEN, cv.WINDOW_FULLSCREEN)

In [2]:
pip install VideoCapture

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\pawls-0.0.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\pdfminer-20191125-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: Could not find a version that satisfies the requirement VideoCapture (from versions: none)
ERROR: No matching distribution found for VideoCapture


In [19]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) 

RuntimeError: Failed to parse: node {
  calculator: "ImagePropertiesCalculator"
  input_stream: "IMAGE:image"
  output_stream: "SIZE:image_size"
}
node {
  calculator: "PreviousLoopbackCalculator"
  input_stream: "MAIN:image"
  input_stream: "LOOP:face_rects_from_landmarks"
  output_stream: "PREV_LOOP:prev_face_rects_from_landmarks"
  input_stream_info {
    tag_index: "LOOP"
    back_edge: true
  }
}
node {
  calculator: "GateCalculator"
  input_stream: "prev_face_rects_from_landmarks"
  output_stream: "gated_prev_face_rects_from_landmarks"
  input_side_packet: "ALLOW:use_prev_landmarks"
  options {
    [mediapipe.GateCalculatorOptions.ext] {
      allow: true
    }
  }
}
node {
  calculator: "NormalizedRectVectorHasMinSizeCalculator"
  input_stream: "ITERABLE:gated_prev_face_rects_from_landmarks"
  output_stream: "prev_has_enough_faces"
  input_side_packet: "num_faces"
}
node {
  calculator: "GateCalculator"
  input_stream: "image"
  input_stream: "DISALLOW:prev_has_enough_faces"
  o

: 